In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

plt.style.use('_mpl-gallery-nogrid')

figs_path = Path('/home/dev/masters-thesis/tex/figs')

path = '/home/dev/masters-thesis/documents/QnA_OSM.xlsx'

questions_df = pd.read_excel(path, sheet_name='Questions').dropna(how='all')
tests_df = pd.read_excel(path, sheet_name='Tests').dropna(how='all')
levels_df = pd.read_excel(path, sheet_name='Prompt Levels').dropna(how='all')
levels_tests_df = pd.read_excel(path, sheet_name='PL Tests').dropna(how='all')

tests_df = tests_df.merge(
    questions_df[['Query ID', 'Difficulty']], on='Query ID', how='left')
levels_tests_df = levels_tests_df.merge(levels_df, on='Query ID', how='left')

In [25]:
tests_df.head()

,Query ID,Agent Type,Outcome,Duration [s],Tokens,Cost [$],LangSmith Traces,Comments,Model,Difficulty
0,aker_brygge_national,oaf,partial success,87.16,3347,0.03465,f1b6c899-da9b-456a-90b1-94e1613316fa,/home/dev/master-thesis/tex/figs/aker_national...,gpt-4-turbo-2024-04-09,1
1,aker_brygge_national,oaf,partial success,97.85,3042,0.03102,98c4beff-bd48-4eb4-b172-495d3d0ae5d2,NaN,gpt-4-turbo-2024-04-09,1
2,aker_brygge_national,oaf,partial success,70.36,4563,0.05095,400c1a15-a5fe-4ca5-a1e2-fb2bedbe807c,NaN,gpt-4-turbo-2024-04-09,1
3,aker_brygge_national,python,partial success,68.10,4736,0.05222,55dcebae-1890-4ef1-8d81-6eead5705730,/home/dev/master-thesis/tex/figs/aker_national...,gpt-4-turbo-2024-04-09,1
4,aker_brygge_national,python,partial success,49.48,2192,0.02226,f6ad6dfc-6a2f-4178-bc73-44d8f8bfb9c9,NaN,gpt-4-turbo-2024-04-09,1


In [45]:
tests_df['Cost [$]'].mean() / (tests_df['Tokens'].mean() / 10**6)

10.687544996400286

In [42]:
import pandas as pd

# Assuming tests_df is your DataFrame and it has been loaded already
# Calculate Q1, Q3, and IQR for each group
Q1 = tests_df.groupby('Agent Type')['Duration [s]'].transform(
    lambda x: x.quantile(0.25))
Q3 = tests_df.groupby('Agent Type')['Duration [s]'].transform(
    lambda x: x.quantile(0.75))
IQR = Q3 - Q1

# Define lower and upper bounds to identify outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(upper_bound)

# Filter out the outliers directly using the bounds calculated for each group
filtered_df = tests_df[(tests_df['Duration [s]'] >= lower_bound) & (
    tests_df['Duration [s]'] <= upper_bound)]

# Calculate the third quartile (Q3) of the filtered data
non_outlier_Q3 = filtered_df.groupby(
    'Agent Type')['Duration [s]'].quantile(.5)

# Print or return the result
print(non_outlier_Q3)

0       186.12500
1       186.12500
2       186.12500
3      1052.18125
4      1052.18125
          ...    
103    1052.18125
104    1052.18125
105     151.19000
106     151.19000
107     151.19000
Name: Duration [s], Length: 108, dtype: float64
Agent Type
oaf       67.410
python    81.580
sql       47.475
Name: Duration [s], dtype: float64


In [ ]:
custom_mapping = {
    'success': 2,
    'partial success': 1,
    'failure': 0
}

tests_df['Outcome_encoded'] = tests_df['Outcome'].map(custom_mapping)

correlation_matrix = tests_df[['Outcome_encoded',
                               'Tokens', 'Duration [s]', 'Cost [$]']].corr()

In [ ]:
import pandas as pd

stddev_by_group = tests_df.groupby(['Query ID', 'Agent Type'])[
    'Outcome_encoded'].std()
mean_stddev_by_agent_type = stddev_by_group.groupby('Agent Type').mean()


print(mean_stddev_by_agent_type)

mean_stddev_by_agent_type.mean()

Agent Type
oaf       0.551567
python    0.336788
sql       0.336788
Name: Outcome_encoded, dtype: float64


0.4083807200603269